## Script for prioritized MAPF

---


In [1]:
import subprocess
import sys
import re

These are the modules that are needed in order to succesfully execute the code below. **```subprocess```** is used in order to make the systemcalls that execute the solver and other management calls like *rm* and *cp*. **```re```** is a very important module that lets us search through the ```.lp```-files to determine robots and destination with the use of regular expressions (regex). Additionally it lets us ultimately convert the output plan into adequate input for further solving steps.

---

In [2]:
target = sys.argv[1]
if target == "-f":
    sys.exit('No target instance defined!')
print(target)

subprocess.run(["conda", "activate", "asprin"])

SystemExit: No target instance defined!

/home/tim/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


When calling the python-script the given argument will be the instance we want to be solved. It throws an exception if no target has been given.

---

In [3]:
non_base_lines   = re.compile("init\(object\(destination.*$" )
where_robots     = re.compile("init\(object\(robot,[0-9]\), value\(at,")
where_dests      = re.compile("init\(object\(destination,[0-9]\), value\(at,")
find_plans       = re.compile("occurs\(object\(robot,")

Here are the regex that are used further down to:
* eliminate lines in the instance that are not required to model the environment. (i.e. robots, destinations, etc.)
* find the robots initialization points (and identifiers).
* find the destinations initialization points (and identifiers).
* convert the plan format into an acceptable input format.
---

In [4]:
robot_idents     = []       # will save the identifiers for the robots
dest_idents      = []       # will save the identifiers for the destinations
dest_inits       = []       # will save the identifiers for the destinations

Theese arrays will hold the lines that save the robots identifiers, ... (duh!). These are the ones that determine the order/priority with which the instance is solved.

---

In [5]:
target_instance  = open("firsttestinstancemd.lp")       # this will be target | target_instance  = open(target) should work                       
temp_instance    = open("temp_instance.lp", "w+")       # a new file "temp_instance.lp" will be created

The ```temp_instance``` will iterate through the number of robots (and destinations) and will gather the respective plans as new inputs. Before the iteration of solving, this instance will represent a base instance that only describes the environment.

---

In [6]:
for line in target_instance.readlines():
    
    #begin# find robot identifiers and initial starting points #######
                                                                    ##
    for match in re.finditer(where_robots, line):                   ##
        robot_idents.append(line[18:-21])                           ##
                                                                    ##
    ##end## find robot identifiers and initial starting points #######
    
    #begin# find destination identifiers and initial position #######
                                                                   ##
    for match in re.finditer(where_dests, line):                   ##
        dest_inits.append(line)                                    ##
        dest_idents.append(line[24:-21])                           ##
                                                                   ##    
    ##end## find destination identifiers and initial position #######
    
    #begin# build base instance (without robot or destination declarations) #######
                                                                                 ##    
    line = re.sub(non_base_lines, "", line)                                      ##
    temp_instance.write(line)                                                    ##
                                                                                 ##
    ##end## build base instance (without robot or destination declarations) #######

target_instance.close()
temp_instance.close()

print(robot_idents)
print(dest_idents)
print(dest_inits)

['1', '2', '3']
['1', '2', '3']
['init(object(destination,1), value(at,(1,1))).\n', 'init(object(destination,2), value(at,(1,2))).\n', 'init(object(destination,3), value(at,(1,7))).\n']


This section uses some regex described above to find the robots and destinations identifiers and initial positions. In the end we delete all non-essential lines for describing the environment.

---

In [9]:
def solve_for_robot_i(i):
    
    #begin# extending temp_instance.lp by destination declaration for robot i ########        
                                                                                    ##
    index = robot_idents.index(i)                                                   ##
    temp_instance = open("temp_instance.lp", "a")                                   ##
    temp_instance.write(dest_inits[index])  # write destination initialization      ##
    temp_instance.close()                                                           ##
                                                                                    ##
    ##end## extending temp_instance.lp by destination declaration for robot i ########           
    print("extended by destination", i)
    
    ####### solve for robot i and its respective destination. Save the output in temp_plan.lp   
    subprocess.run("clingo ./testsolver.lp temp_instance.lp > temp_plan.lp --outf=1 --out-atomf='%s.' --out-ifs='\n' --time-limit=10", shell=True)    
    
    print("solved for destination", i)

    temp_instance = open("temp_instance.lp", "r")       # alter this instance to contain a converted plan for further solving
    lines = temp_instance.readlines()
    temp_instance.close()
    
    temp_instance = open("temp_instance.lp", "w")
    for line in lines:
        if line.strip("\n") != dest_inits[index][:-1]:
            temp_instance.write(line)
    
    #begin# extending temp_instance.lp by the plan for robot i
    
    # occurs(object(robot,1),action(move,(0,-1)),2)
    # position(     robot(R),            (X, Y) ,T)
    
    plan_instance = open("temp_plan.lp", "r+")          # read from plan-instance
    
    for line in plan_instance.readlines():
        for match in re.finditer(find_plans, line):
            print("occurs found")
            temp_instance.write(line)
    
    ### WRITE CODE THAT CLEANS UP temp_plan.lp TO ONLY CONTAIN occurs-STATEMENTS (they all end with ")." )
    
        
    print("extended for plan to destination", i)
            
    plan_instance.close()        
    temp_instance.close()

**This is the heart-piece of the script.**

This section is divided into three parts that allow for an iterative solving process.
### Part I
simply extends the current iteration of the instance by the robot and destination locations and identifiers that have been extracted previously.

### Part II
is the execution of the embedded solver. Note that this solver only solves for one robot at the time! Therefore this solver could be replaced by any optimal single agent solver. The plan is saved in a file ```temp_plan.lp```

### Part III
extends the temporary instance by the ```occurs```-statements that are used within the dl-solver.

In [10]:
for identifier in robot_idents:
    print("solving for robot", identifier)
    solve_for_robot_i(identifier)
    break

solving for robot 1
extended by destination 1
solved for destination 1
extended for plan to destination 1


This iterates through the array of robot identifiers and solves the instance with their respective destination.

---

In [68]:
subprocess.run(["rm", "temp_instance.lp"])
subprocess.run(["cp", "temp_plan.lp", "target_plan.lp"])
subprocess.run(["rm", "temp_plan.lp"])

CompletedProcess(args=['rm', 'temp_plan.lp'], returncode=0)

This deletes the files that are no longer needed. ```target_plan.lp``` can later be replaced by a given argument when calling the script.

---